In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, LayerNormalization, Dropout
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

2024-12-29 20:43:02.232623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735485182.329768 1387942 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735485182.358839 1387942 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-29 20:43:02.592279: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with open('training_data.txt', 'r', encoding='utf-8') as f:
    data = f.read().replace('\n', ' ')

In [3]:
print(len(data))

1115394


In [4]:
characters=list(set(list(data)))
print(len(characters))

64


In [5]:
character_to_integer_encoding={}
integer_to_character_encoding={}
for i in range(len(characters)):
    character_to_integer_encoding[characters[i]]=i+1
    integer_to_character_encoding[i+1]=characters[i]

In [6]:
def encode(string):
    global character_to_integer_encoding    
    return [character_to_integer_encoding[char] for char in string]

def decode(lst):
    global integer_to_character_encoding
    return ''.join([integer_to_character_encoding[i] for i in lst])

In [7]:
input_data=encode(data)
train_data=input_data[:int(0.9*len(input_data))]
test_data=input_data[int(0.9*len(input_data)):]

In [8]:
batch_size=32
block_size=128
num_heads=8 # Experiment with other values if you want
num_transformer_blocks = 4
input_vocab_size=len(characters)+1
feed_forward_dim = 256 # I am using the same dimensions for the embedding as well. This may be too high of a dimension, given that there are only 65 characters and 128 positions per block, but it will take a lot of time to test alternate parameters

In [ ]:
'''
Implementing the Multihead attention layer was something I tried, 
but ultimately it gave slower and worse results than calling layers.MultiHeadAttention 
(ig the people at tensorflow have put some effort into optimization). 
You can try modifying the code in this cell and using it instead of calling the inbuilt class
'''
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, model_dimension):
        super().__init__()
        self.num_heads = num_heads    
        self.model_dimension = model_dimension
        assert model_dimension % num_heads == 0

        self.depth = model_dimension // num_heads
        self.query_space_projector = Dense(model_dimension)
        self.key_space_projector = Dense(model_dimension)
        self.value_space_projector = Dense(model_dimension)
        self.dense = Dense(model_dimension)
    
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])  # Shape: (batch_size, num_heads, seq_len, depth)

    def causal_attention_mask(self, batch_size, n_dest, n_src):
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j  
        mask = tf.cast(m, tf.bool)
        mask = tf.reshape(mask, [1, n_dest, n_src])  # Shape: (1, n_dest, n_src)
        mask = tf.tile(mask, [batch_size, 1, 1])  # Shape: (batch_size, n_dest, n_src)
        mask = mask[:, tf.newaxis, :, :]  # Shape: (batch_size, 1, n_dest, n_src)
        mask = tf.tile(mask, [1, self.num_heads, 1, 1])  # Shape: (batch_size, num_heads, n_dest, n_src)
        return mask

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        q = self.query_space_projector(inputs)
        k = self.key_space_projector(inputs)
        v = self.value_space_projector(inputs)
        
        q = self.split_heads(q, batch_size)  # Shape: (batch_size, num_heads, seq_len, depth)
        k = self.split_heads(k, batch_size)  # Shape: (batch_size, num_heads, seq_len, depth)
        v = self.split_heads(v, batch_size)  # Shape: (batch_size, num_heads, seq_len, depth)

        mask = self.causal_attention_mask(batch_size, tf.shape(inputs)[1], tf.shape(inputs)[1])

        mask = tf.cast(mask, tf.float32)

        attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask)

        attention = tf.transpose(attention, perm=[0, 2, 1, 3])  # Shape: (batch_size, seq_len, num_heads, depth)

        attention = tf.reshape(attention, (batch_size, -1, self.model_dimension))  # Shape: (batch_size, seq_len, model_dimension)

        output = self.dense(attention)
        return output

    def scaled_dot_product_attention(self, q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)  # Shape: (batch_size, num_heads, seq_len, seq_len)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        scaled_attention_logits += (mask * -1e9)  

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

        output = tf.matmul(attention_weights, v)  # Shape: (batch_size, num_heads, seq_len, depth)
        return output, attention_weights

In [9]:
def causal_attention_mask(batch_size, n_dest, n_src):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, tf.bool)
    mask = tf.reshape() ### -> Fill this up
    return tf.tile(mask, [batch_size, 1, 1])


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        # Give code for an attention layer, feedforward layers, and normalization layers. The attention layer is first, then normalization and dropout, then forward the data passed through a non-linear function, and call the dropout layer again
        ###
        # Insert code here
        ###
        self.normalization_layer_1 = LayerNormalization(epsilon=1e-6)
        self.normalization_layer_2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        block_size = input_shape[1]
        ''' Insert the calling code here '''
        return 

In [10]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_embedding = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_embedding = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        ### -> Fill this up


In [11]:
def get_transformer_model(
    maxlen, 
    vocab_size, 
    embed_dim, 
    num_heads, 
    feed_forward_dim, 
    num_transformer_blocks=1
):
    inputs = Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    for _ in range(num_transformer_blocks):
        transformer_block = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
        x = transformer_block(x)
    outputs = Dense(vocab_size)(x)
    model = Model(inputs=inputs, outputs=[outputs]) # This is a functional API-based representation of a tf model
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        "adam", 
        loss=[loss_fn],
        metrics=["accuracy"]
    ) 
    return model

In [12]:
model = get_transformer_model(block_size, input_vocab_size, feed_forward_dim, num_heads, feed_forward_dim, num_transformer_blocks)


2024-12-29 20:43:08.870783: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [13]:
inputs = [train_data[i:i+block_size] for i in range(0, len(train_data)-block_size-1)]
targets = [train_data[i+1:i+block_size+1] for i in range(0, len(train_data)-block_size-1)]

'''
Insert code here to preprocess the input data and the target data to send it to the model.
'''

dataset= tf.data.Dataset.from_tensor_slices((inputs, targets))
dataset = dataset.shuffle(10000)
dataset = dataset.batch(batch_size, drop_remainder=True)

2024-12-29 20:43:31.089570: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1027814400 exceeds 10% of free system memory.
2024-12-29 20:43:32.144429: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1027814400 exceeds 10% of free system memory.


In [14]:
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, 128, 256)       │        49,408 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 128, 256)       │     2,236,160 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_1             │ (None, 128, 256)       │     2,236,160 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_2             │ (None, 128, 256)       │     2,236,160 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_3             │ (None, 128, 256)       │     2,236,160 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128, 65)        │        16,705 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,010,753 (34.37 MB)

 Trainable params: 9,010,753 (34.37 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
'''
It will take a long time for the entirety of this function to run. However, you can always stop execution after short durations to evaluate how the code is performing. The result in the bottommost cell is after training on only 816 out of a potential 313660 batches.
The dataset has to be shuffled between each time you call this cell to avoid running the model only on the first few input-target pairs multiple times, which may cause you to think the model is performing better than it actually is.
'''
dataset= tf.data.Dataset.from_tensor_slices((inputs, targets))
dataset=dataset.shuffle(100)
dataset = dataset.batch(batch_size, drop_remainder=True)
model.fit(dataset, epochs=10)

Epoch 1/10


2024-12-29 20:43:40.440453: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1027814400 exceeds 10% of free system memory.
2024-12-29 20:43:40.873963: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1027814400 exceeds 10% of free system memory.


  816/31366 ━━━━━━━━━━━━━━━━━━━━ 21:45:32 3s/step - dense_8_accuracy: 0.3675 - loss: 2.2600

KeyboardInterrupt: 

In [17]:
def generate_text(model, start_index, num_generate=1):
    '''
        This function will generate text for num_generate characters, starting from start_index+batch_size.
    '''
    input_sequence = train_data[start_index:start_index + block_size]
    generated_text = decode(input_sequence)
    for i in range(num_generate):
        input_eval = tf.convert_to_tensor([input_sequence], dtype=tf.int32)
        predictions, _ = model.predict(input_eval)
        probabilities = tf.nn.softmax(predictions[0, -1]).numpy()
        next_token = np.random.choice(len(probabilities), p=probabilities)
        input_sequence += [next_token]
        input_sequence = input_sequence[1:]
        generated_text += decode([next_token])

    return generated_text


In [19]:
generate_text(model, start_index=0, num_generate=1000)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━

"First Citizen: Before we proceed any further, hear me speak.  All: Speak, speak.  First Citizen: You are all resolved rather to bend! C COMINIUS: O, we word me? I were the flies, We the were the the warrse aff-a The like bly we he cair dused, He like one upone, By freas ince swere we wi n hare ceant, Being forthern fing woules and dear sword, The citiven, tride bain gin the firest: Heven die lad 'end cition, Seeed belies the pacelt har.  COMINIUS: The cuset ire the business.  BRUTUS: O deverr good.  MARCIUS: Let he he rest bust e nor: themst deemve ber'd forth, where cinsed.  VIRTILIA: He's Befir, bett I do beceech meain y reed, wo'll not.  COMINIUS: Too metuar is, anded.  MARCIUS: Be the we firculd the ven The Your four the for gentl's: The var the Apon equart moors fe force, True sword, the pacians, and fall foust, Wher the dove let for cus than hougght, we ca fret god. Whip or gites, ffor ust: to the hir best to ver.  MARCIUS: He's the is Te bellust he peroper intly: be ecared, Tha